In [66]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import OneHotEncoder

In [67]:
anime_df = pd.read_csv('24_anime/anime.csv')
rating_df = pd.read_csv('24_anime/rating.csv')

In [68]:
## DROP RATING, NAME AND MEMBERS DUE TO INSIGNIFICANCE ##
anime_df = anime_df.drop(['rating','name','members'], axis=1)

## REMOVE -1 FROM RATINGS ##
rating_df = rating_df[rating_df.rating != -1]

## REMOVE UKNOWN FROM NUMBER OF EPISODES

In [69]:
df_merge = pd.merge(rating_df, anime_df, on=['anime_id'])

In [70]:
df = df_merge
column_list = df.columns.values.tolist()

df.replace('nan', np.nan, inplace=True)
df.replace('', pd.NA, inplace=True)
df = df.dropna()
df = df[df.episodes != 'Unknown']
df = df.astype({'episodes':'float64'})


df = df.dropna(how='any')           # assign back
df.dropna(how='any', inplace=True)  # set inplace parameter

for col in df:
    print(df[col].unique())

print(df.head())

[    1     3     5 ... 66261 68269 73345]
[ 8074 11617 11757 ... 30738  8723  8749]
[10  6  2  7  9  8  4  5  3  1]
['Action, Ecchi, Horror, Supernatural'
 'Comedy, Demons, Ecchi, Harem, Romance, School'
 'Action, Adventure, Fantasy, Game, Romance' ...
 'Adventure, Shoujo, Slice of Life'
 'Action, Adventure, Kids, Mecha, Sci-Fi, Space'
 'Action, Historical, Kids']
['TV' 'Movie' 'OVA' 'Special' 'ONA' 'Music']
[1.200e+01 2.500e+01 2.200e+02 6.400e+01 1.010e+02 1.000e+00 2.760e+02
 5.400e+01 2.910e+02 5.000e+01 5.000e+00 3.700e+01 1.920e+02 3.000e+00
 1.100e+01 5.100e+01 1.270e+02 1.750e+02 8.400e+01 4.700e+01 1.300e+01
 2.400e+01 4.300e+01 4.000e+00 2.600e+01 1.450e+02 5.200e+01 1.780e+02
 9.400e+01 2.300e+01 4.900e+01 6.200e+01 8.000e+00 1.400e+01 1.530e+02
 7.000e+01 1.670e+02 1.500e+02 7.500e+01 6.000e+00 3.660e+02 2.000e+01
 1.120e+02 2.000e+00 2.240e+02 1.800e+02 2.960e+02 2.700e+01 2.010e+02
 1.140e+02 4.800e+01 1.280e+02 1.600e+01 4.500e+01 6.600e+01 3.900e+01
 1.000e+01 2.200e+01

In [71]:
def one_hot_encode_single_values(df, column_name):
    # Extract the column as a Series
    column = df[column_name]

    # Reshape the column as a 2D array
    reshaped_column = column.values.reshape(-1, 1)

    # Initialize OneHotEncoder and fit it to the column
    encoder = OneHotEncoder(sparse=False)
    one_hot_encoded = encoder.fit_transform(reshaped_column)

    # Create a new DataFrame with the one-hot encoded values
    one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.categories_[0])
    print(encoder.categories_[0])

    # Concatenate the original DataFrame with the one-hot encoded DataFrame
    df = pd.concat([df, one_hot_df], axis=1)

    return df

def one_hot_encode_dataframe(df, column_name):
    # Split values in the specified column using commas
    df[column_name] = df[column_name].str.replace(' ','')
    df[column_name] = df[column_name].str.split(',')
    

    print(df[column_name])

    # Initialize MultiLabelBinarizer and fit it to the column
    mlb = MultiLabelBinarizer()
    one_hot_encoded = mlb.fit_transform(df[column_name])
    # classes = mlb.classes_

    # Create a new DataFrame with the one-hot encoded values
    one_hot_df = pd.DataFrame(one_hot_encoded, columns=mlb.classes_)

    # Concatenate the original DataFrame with the one-hot encoded DataFrame
    df = pd.concat([df, one_hot_df], axis=1)

    return df

In [72]:
df = one_hot_encode_dataframe(df, 'genre')

df = one_hot_encode_single_values(df, 'type')

df = df.dropna()

df.to_csv('data.csv', header=True, index=False)

Before:
0          Action,Ecchi,Horror,Supernatural
1          Action,Ecchi,Horror,Supernatural
2          Action,Ecchi,Horror,Supernatural
3          Action,Ecchi,Horror,Supernatural
4          Action,Ecchi,Horror,Supernatural
                         ...               
6337234         Adventure,Drama,SliceofLife
6337235            Comedy,Magic,SliceofLife
6337236                      Adventure,Kids
6337237                              Comedy
6337238                              Comedy
Name: genre, Length: 6337145, dtype: object
After:
0          [Action, Ecchi, Horror, Supernatural]
1          [Action, Ecchi, Horror, Supernatural]
2          [Action, Ecchi, Horror, Supernatural]
3          [Action, Ecchi, Horror, Supernatural]
4          [Action, Ecchi, Horror, Supernatural]
                           ...                  
6337234          [Adventure, Drama, SliceofLife]
6337235             [Comedy, Magic, SliceofLife]
6337236                        [Adventure, Kids]
6337237         